In [1]:
if __name__ == '__main__':
    print ("^.^")
else:
    print (">.<")

^.^


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.conf import SparkConf
from pyspark.sql import Row

import random 

from matplotlib import pyplot as plta
%matplotlib inline
from plotly import graph_objects as go

from scapy.all import *

import sys
import os
import psutil

import time
import json

In [3]:
spark = (SparkSession.builder
                     .config('spark.app.name', 'MAIN')
                     .config('spark.driver.memory', '16g')
                     .config('spark.driver.cores', '2')
                     .config('spark.executor.cores', '4')
                     .config('spark.executor.memory', '34g')
                     .config('spark.executor.instance', '2')
                     .config('spark.dynamicAllocation.enabled', False)
                     .config('spark.shuffle.service.enabled', False)
                     .config('spark.rdd.compress', False)
                     .config('spark.sql.repl.eagerEval.enabled', True)
                     .config('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')
                     .config('spark.driver.extraClassPath', '/home/kei/.local/lib/python3.8/site-packages/sparkmonitor/listener_2.12.jar')
                     #.config('spark.delight.accessToken.secret', 'f463ad09da4d1582f4508b7e3c894e90efc5777c00985965976801cc96a9b2b5')
                     #.config('spark.delight.appNameOverride', 'spark.app.name')
                     .getOrCreate())

sc = spark.sparkContext

sparkConf = SparkConf().getAll()


sc, spark, sparkConf[:]

(<SparkContext master=local[*] appName=MAIN>,
 [('spark.executor.memory', '34g'),
  ('spark.shuffle.service.enabled', 'False'),
  ('spark.sql.repl.eagerEval.enabled', 'True'),
  ('spark.driver.cores', '2'),
  ('spark.executor.instance', '2'),
  ('spark.app.submitTime', '1666032160740'),
  ('spark.driver.memory', '16g'),
  ('spark.rdd.compress', 'False'),
  ('spark.executor.cores', '4'),
  ('spark.dynamicAllocation.enabled', 'False'),
  ('spark.app.name', 'MAIN'),
  ('spark.driver.extraClassPath',
   '/home/kei/.local/lib/python3.8/site-packages/sparkmonitor/listener_2.12.jar'),
  ('spark.extraListeners',
   'sparkmonitor.listener.JupyterSparkMonitorListener'),
  ('spark.master', 'local[*]'),
  ('spark.submit.pyFiles', ''),
  ('spark.submit.deployMode', 'client'),
  ('spark.ui.showConsoleProgress', 'true')])

In [3]:
b=sc.broadcast([3,10]) #Creating a broadcast variable available on all executors
a=sc.accumulator(0)   #Creating an accumulator for adding values across executors
RDD0=sc.parallelize([y for y in range(0,5)]) #RDD from input python collection
RDD2=sc.parallelize([z for z in range(1, 10)])
RDD1=RDD0.cartesian(RDD2)
cached=RDD2.cache() #Testing cached RDD
RDD22=RDD1.map(lambda x:x[0]+x[1]+b.value[0])
RDD3=RDD22.repartition(5) # To trigger a new stage.
RDD4=RDD2.map(lambda x: 3*x-b.value[0])
RDD5=RDD3.filter(lambda x:x%2==0)
RDD6=RDD4.filter(lambda x:x%2!=0)
RDD7=RDD5.cartesian(RDD6)
RDD8=RDD7.flatMap(lambda x: [x[i] for i in range(0,2)])
RDD9=RDD8.union(cached)
ans=RDD9.reduce(lambda x,y: x+y) # Doing a simple sum on the random data.
print(ans)
def f(x):
    global a
    time.sleep(0.58) #Making the job run a little longer
    a+=x
RDD9.foreach(f)
print(a.value)
#Display should appear automatically

2069
2069


In [4]:
mainFolder = "/home/kei/Desktop/data/innostage/innostage_task_3/pcap_data/capture_packets/"

filesList = []
for file in os.listdir(mainFolder):
    pcapFolder = '/home/kei/Desktop/data/innostage/innostage_task_3/pcap_data/capture_packets/{0}'.format(file)
    filesList.append(pcapFolder)
    

In [5]:
#Таблица признаков и их описание
F_columns = ['№', 'Признак', 'Описание']
F_data = [(1, 'Размер запроса в байтах', 'Отклонение от средних показателей 40-60 байт в сторону увеличения может говорить о скрытом канале'),
          (2, 'Необычный тип трафика', 'Если в трафике используются данные вида TXT, NULL, KEY - это говорит об аномалии'),
          (3, 'Длина имени запроса', 'Имена используемые злоумышленниками могут достигать 200 символов - средняя 30 символов')]

In [6]:
feauture_DF = spark.createDataFrame(F_data, F_columns)

In [7]:
feauture_DF

№,Признак,Описание
1,Размер запроса в ...,Отклонение от сре...
2,Необычный тип тра...,Если в трафике ис...
3,Длина имени запроса,Имена используемы...


In [9]:
def extract_data(files):
    data = []

    for file in files:
        
        for pkt in PcapReader(file):
            
            if pkt.haslayer(DNSQR):
                query = pkt.getlayer(DNSQR)
                
                qtype_field = query.get_field('qtype')
                qtype_data = qtype_field.i2repr(query, query.qtype)
                
                qname_field = query.get_field('qname')
                qname_data = qname_field.i2repr(query, query.qname)
                
                data.append((qtype_data, qname_data.__len__(), pkt.len))
                
    return data

In [10]:
def get_run_time(startTime):
    return time.time() - startTime

In [4]:
dfSchema = StructType([StructField('TrafficType', StringType(), True),
                             StructField('NameLenght', IntegerType(), True),
                             StructField('Bytes', IntegerType(), True)])
dfSchema

StructType([StructField('TrafficType', StringType(), True), StructField('NameLenght', IntegerType(), True), StructField('Bytes', IntegerType(), True)])

In [41]:
#data = extract_data(filesList)

In [42]:
#with open('data.txt', 'w') as file_write:
#    json.dump(data, file_write)

In [43]:
with open('/home/kei/Desktop/data/innostage/innostage_task_3/extracted_data/data.txt', 'r') as file_read:
    data = json.load(file_read)

In [44]:
main_DF = spark.createDataFrame(sc.parallelize(data), dfSchema)

In [45]:
main_DF.show()

+-----------+----------+-----+
|TrafficType|NameLenght|Bytes|
+-----------+----------+-----+
|          A|        24|   67|
|       AAAA|        24|   67|
|          A|        18|   61|
|       AAAA|        18|   61|
|       AAAA|        22|   65|
|          A|        22|   65|
|       AAAA|        22|   65|
|          A|        22|   65|
|       AAAA|        22|  122|
|       AAAA|        22|  122|
|          A|        22|   81|
|          A|        22|   81|
|          A|        24|   67|
|       AAAA|        24|   67|
|        PTR|        27|   70|
|        PTR|        27|  105|
|          A|        18|   61|
|       AAAA|        18|   61|
|          A|        15|   58|
|          A|        15|  223|
+-----------+----------+-----+
only showing top 20 rows



In [46]:
#main_DF.write.parquet('/home/kei/Desktop/data/docs/PARQUET/traffic_data')

In [21]:
fileParquet = spark.read.parquet('/home/kei/Desktop/data/innostage/innostage_task_3/parquet_data/traffic_data_rdy/')

______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-

# #TODO 
    - Подогнать набор данных так, чтобы результаты обработки максимально соответствовали конфигурации

In [5]:
#Processing Data Check.
#DATA_PROCESSING_RESULT = []
data_for_shit = []


parquetPath = '/home/kei/Desktop/data/innostage/innostage_task_3/parquet_data/'

field = parquetPath + 'traffic_data_field1'

with open('/home/kei/Desktop/data/innostage/innostage_task_3/extracted_data/data.txt', 'r') as file_read:
    data = json.load(file_read)


for i in range(1, 2):
    
    b=sc.broadcast([3,10]) #Creating a broadcast variable available on all executors
    a=sc.accumulator(0)   #Creating an accumulator for adding values across executors
    RDD0=sc.parallelize([y for y in range(0,7)]) #RDD from input python collection
    RDD2=sc.parallelize([z for z in range(1, 10)])
    RDD1=RDD0.cartesian(RDD2)
    cached=RDD2.cache() #Testing cached RDD
    RDD22=RDD1.map(lambda x:x[0]+x[1]+b.value[0])
    RDD3=RDD22.repartition(5) # To trigger a new stage.
    RDD4=RDD2.map(lambda x: 3*x-b.value[0])
    RDD5=RDD3.filter(lambda x:x%2==0)
    RDD6=RDD4.filter(lambda x:x%2!=0)
    RDD7=RDD5.cartesian(RDD6)
    RDD8=RDD7.flatMap(lambda x: [x[i] for i in range(0,2)])
    RDD9=RDD8.union(cached)
    ans=RDD9.reduce(lambda x,y: x+y) # Doing a simple sum on the random data.
    print(ans)
    def f(x):
        global a
        time.sleep(0.085) #Making the job run a little longer
        a+=x
    RDD9.foreach(f)



main_DF = spark.createDataFrame(sc.parallelize(data_for_shit), dfSchema)
#main_DF.write.parquet(field)


fileParquet = spark.read.parquet(field)
#---------------------------------------------------------------------------

#with open('process_result_field1.txt', 'w') as file_write:
#    json.dump(DATA_PROCESSING_RESULT, file_write)
#

2989


______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-______-

In [26]:
processingTestSchema = ['Filed', 'Driver Memory', 'Executor Memory', 'Executor Instance', 'Driver Cores', 'Processing Time', 'Bytes']

In [68]:
#json_data = []
#for eachFile in os.listdir('/home/kei/Desktop/data/docs/JSON/'):
#    
#    with open(str(eachFile), 'r') as file_read:
#        json_data.append(json.load(file_read))
#
#        
#jsonAggregatedData = []
#for i in json_data:
#    for k in i:
#        jsonAggregatedData.append(tuple(k))


#with open('json_data.txt', 'w') as file:
#    json.dump(json_data, file)

In [27]:
test_DF = spark.createDataFrame(data, processingTestSchema)

IndexError: list index out of range

In [55]:
#Processing Data Check.
#DATA_PROCESSING_RESULT = []
#parquetPath = 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PARQUET\\'
#
#--------------------------------------------------------------------------
#
#field: 1.
#startTime = time.time()
#
#field_1 = parquetPath + 'traffic_data_field1\\'
#
#with open('C:/Users/igor.beryozkin/Desktop/docs/TXT/innostage_task3/extracted_data/data.txt', 'r') as file_read:
#   data = json.load(file_read)
##
#
#main_DF = spark.createDataFrame(sparkContext.parallelize(data), dfSchema)
#    
#main_DF.write.parquet(field_1)
#fileParquet = spark.read.parquet(field_1)
#
#DATA_PROCESSING_RESULT.append(('field 1', SparkConf().get('spark.driver.memory'),
#                                SparkConf().get('spark.executor.memory'),
#                                SparkConf().get('spark.executor.instance'),
#                                SparkConf().get('spark.driver.cores'),
#                                get_run_time(startTime),
#                                get_path_size_Bytes(field_1)))
#
#---------------------------------------------------------------------------
#
#with open('process_result_field1.txt', 'w') as file_write:
#    json.dump(DATA_PROCESSING_RESULT, file_write)
#